In [5]:
# ----------------------------------------------- #
#                      Setup                      #
# ----------------------------------------------- #
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
kerasBKED = os.environ["KERAS_BACKEND"] 

import keras
from keras.models import load_model
from keras.datasets import cifar10
from keras.datasets import mnist
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation, LeakyReLU
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/pywrap_tensorflow.py", line 41, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/usr/lib/python3.5/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/usr/lib/python3.5/imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: libcusolver.so.8.0: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/install_sources#common_

In [ ]:
# ----------------------------------------------- #
#                    Load Data                    #
# ----------------------------------------------- #
batch_size = 32
num_classes = 10
epochs = 10

saveDir = "./ConvAEFiles/"
if not os.path.isdir(saveDir):
    os.makedirs(saveDir)

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# normalize data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# divide x_test into validation and test
x_val = x_test[:7000]
x_test = x_test[7000:]

print("validation data: {0} \ntest data: {1}\n".format(x_val.shape, x_test.shape))

In [ ]:
# ----------------------------------------------- #
#                    Model                        #
# ----------------------------------------------- #
input_img = Input(shape=(32, 32, 3))

x = Conv2D(64, (3, 3), activation="relu", padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation="relu", padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation="relu", padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same', name="encoded")(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(16, (3, 3), activation="relu", padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation="relu", padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation="relu", padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Create an instance of the model
model = Model(input_img, decoded)
model.compile(optimizer='adam', loss='binary_crossentropy')
encoder = Model(inputs=model.input, outputs=model.get_layer('encoded').output)
# model.summary()

# Create a decoder model
encoded_input = Input(shape=(4,4,16))
deco = model.layers[-7](encoded_input)
deco = model.layers[-6](deco)
deco = model.layers[-5](deco)
deco = model.layers[-4](deco)
deco = model.layers[-3](deco)
deco = model.layers[-2](deco)
deco = model.layers[-1](deco)
decoder = Model(encoded_input, deco)
# decoder.summary()

In [ ]:
# ----------------------------------------------- #
#                    Train Model                  #
# ----------------------------------------------- #
model.load_weights(saveDir + "AutoEncoder_Cifar10_Deep_weights.10-0.57-0.57.hdf5") 

# Es_cb = EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='auto')
# Chkpt = saveDir + 'AutoEncoder_Cifar10_Deep_weights.{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5'
# Cp_cb = ModelCheckpoint(filepath = chkpt, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
# 
# Story = model.fit(x_train, x_train,
#                   batch_size=batch_size,
#                   epochs=epochs,
#                   verbose=1,
#                   validation_data=(x_val, x_val),
#                   callbacks=[es_cb, cp_cb],
#                   shuffle=True)

print("Finished training")
score = model.evaluate(x_test, x_test, verbose=1)
print(score)

In [ ]:
# ----------------------------------------------- #
#                    Plotting 				      #
# ----------------------------------------------- #
c10test = model.predict(x_test, verbose=1)
c10val = model.predict(x_val, verbose=1)
c10encoded = encoder.predict(x_test, verbose=1)

# definition to show original image and reconstructed image
def showOrigDec(orig, enc, dec, num=10):
    n = num
    plt.figure(figsize=(20, 4))

    for i in range(n):
        # display original
        ax = plt.subplot(3, n, i+1)
        plt.imshow(orig[i].reshape(32, 32, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display encoded
        ax = plt.subplot(3, n, i+1+n)
        plt.imshow(enc[i].reshape(16,16))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        
        # display reconstruction
        ax = plt.subplot(3, n, i+1+n+n)
        plt.imshow(dec[i].reshape(32, 32, 3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.savefig("plt_reconstructed.png")
    plt.show()

# Plot 
showOrigDec(x_test, c10encoded, c10test)
print("Plot saved to plt_reconstructed.png")

In [ ]:
# ----------------------------------------------- #
#          Decoding Normal Distributions          #
# ----------------------------------------------- #
normal_dists = []
for i in range(10):
    normal_dists.append(np.random.normal(1.0, 0.5, 256))
    
decoded_dists = []
for i in range(10):
    decoded_dists.append(decoder.predict(normal_dists[i].reshape(1,4,4,16)))

n = 10
plt.figure(figsize=(20, 4))

for i in range(n):
    # display encoded
    ax = plt.subplot(2, n, i+1)
    plt.imshow(normal_dists[i].reshape(16,16))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i+1+n)
    plt.imshow(decoded_dists[i].reshape(32, 32, 3))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.savefig("plt_decoded_dists.png")
plt.show()
print("Plot saved to plt_decoded_dists.png")
